In [57]:
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from homomesy import *
import matplotlib.pyplot as plt
import numpy as np
from math import comb
findstat()._allow_execution = True


In [ ]:
# credit: chatgpt :)
def can_match_all(elements, *conditions):
    n = len(elements)
    if len(conditions) != n:
        return False

    # Build adjacency list: edges[i] = list of condition indices that element i can satisfy
    edges = [[] for _ in range(n)]
    for i, element in enumerate(elements):
        for j, condition in enumerate(conditions):
            if condition(element):
                edges[i].append(j)

    # match_to_condition[j] = index of element assigned to condition j
    match_to_condition = [-1] * n

    def bpm(u, visited):
        for v in edges[u]:
            if not visited[v]:
                visited[v] = True
                # If condition v is not matched or previous match can be reassigned
                if match_to_condition[v] == -1 or bpm(match_to_condition[v], visited):
                    match_to_condition[v] = u
                    return True
        return False

    result = 0
    for u in range(n):
        visited = [False] * n
        if bpm(u, visited):
            result += 1

    return result == n

# I hope this is correct, I spend way too much time on this.
def orbit_size_formula(n, k):
    if (n // k) * k != n:
        return 0
    elif k == 1:
        return 1
    elif k == 2:
        return n // 2
    elif k == 3:
        return {
            f"(1,1,a); 1<a≤{2*n//3+1}: {2*n // 3}": lambda lst: lst[0] == 1
            and lst[1] == 1
            and 1 < lst[2] <= 2 * n // 3 + 1,
            f"(1,a,a); 1<a≤{n//3 + 1}: {n // 3}": lambda lst: lst[0] == 1
            and lst[1] == lst[2]
            and 1 < lst[1] <= n // 3 + 1,
            f"(1,a,b); 1<a,b≤{n//3 + 1}: {comb(n // 3, 2)*2}": lambda lst: lst[0] == 1
            and lst[1] != lst[2]
            and 1 < lst[1] <= n // 3 + 1
            and 1 < lst[2] <= n // 3 + 1,
            f"(1,a,b); 1<a≤{n//3+1}, {n//3 + 1}<b≤{2*n//3 + 1}: {(n // 3)**2 * 2}": lambda lst: (
                lst[0] == 1
                and lst[1] != lst[2]
                and can_match_all(
                    lst[1:],
                    lambda x: 1 < x <= 2 * n // 3 + 1,
                    lambda x: n // 3 + 1 < x <= 2 * n // 3 + 1,
                )
            ),
            "sum": (2 * n // 3) + (n // 3) + comb(n // 3, 2) * 2 + (n // 3) ** 2 * 2,
        }
    elif k == 4:
        return {
            f"(1,1,1,a); 1<a≤{3*n//4+1}: {3*n // 4}": lambda lst: lst[0] == 1
            and lst[1] == 1
            and lst[2] == 1
            and 1 < lst[3] <= 3 * n // 4 + 1,
            f"(1,1,a,a); 1<a≤{n//2 + 1}: {n // 2}": lambda lst: lst[0] == 1
            and lst[1] == 1
            and lst[2] == lst[3]
            and 1 < lst[2] <= n // 2 + 1,
            f"(1,a,a,a); 1<a≤{n//4 + 1}: {n // 4}": lambda lst: lst[0] == 1
            and lst[1] == lst[2] == lst[3]
            and 1 < lst[1] <= n // 4 + 1,
            f"(1,1,a,b); a≠b, 1<a,b≤{n//2 + 1}: {comb(n // 2, 2)*2}": lambda lst: (
                lst[0] == 1
                and lst[1] == 1
                and lst[2] != lst[3]
                and 1 < lst[2] <= n // 2 + 1
                and 1 < lst[3] <= n // 2 + 1
            ),
            f"(1,1,a,b); a≠b, 1<a≤{n//2 + 1}, {n//2 + 1}<b≤{3*n//4 + 1}: {2*n**2 // 8}": lambda lst: (
                lst[0] == 1
                and lst[1] == 1
                and lst[2] != lst[3]
                and can_match_all(
                    lst[2:],
                    lambda x: 1 < x <= n // 2 + 1,
                    lambda x: n // 2 + 1 < x <= 3 * n // 4 + 1,
                )
            ),
            f"(1,a,b,b); a≠b, 1<a,b≤{n//4 + 1}: {3*comb(n // 4, 2)*2}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 2
                and all(1 < x <= n // 4 + 1 for x in lst[1:])
            ),
            f"(1,a,b,b); a≠b, 1<a≤{n//4 + 1}, {n//4 + 1}<b≤{n//2 + 1}: {3*n**2 // 16}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 2
                and can_match_all(
                    lst[1:],
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: n // 4 + 1 < x <= n // 2 + 1,
                    lambda x: n // 4 + 1 < x <= n // 2 + 1,
                )
            ),
            f"(1,a,b,b); a≠b, {n//4 + 1}<a≤{3*n//4 + 1}, 1<b≤{n//4 + 1}: {3*n**2 // 8}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 2
                and can_match_all(
                    lst[1:],
                    lambda x: n // 4 + 1 < x <= 3 * n // 4 + 1,
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: 1 < x <= n // 4 + 1,
                )
            ),
            f"(1,a,b,c); a≠b≠c≠a, 1<a,b,c≤{n//4 + 1}: {6*comb(n // 4, 3)}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 3
                and all(1 < x <= n // 4 + 1 for x in lst[1:])
            ),
            f"(1,a,b,c); a≠b≠c≠a, 1<a,b≤{n//4 + 1}, {n//4 + 1}<c≤{3*n//4 + 1}: {3*2*comb(n // 4, 2)*(n // 2)}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 3
                and can_match_all(
                    lst[1:],
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: n // 4 + 1 < x <= 3 * n // 4 + 1,
                )
            ),
            f"(1,a,b,c); a≠b≠c≠a, 1<a≤{n//4 + 1}, {n//4 + 1}<b,c≤{n//2 + 1}: {3*(n // 4)*2*comb(n // 4, 2)}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 3
                and can_match_all(
                    lst[1:],
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: n // 4 + 1 < x <= n // 2 + 1,
                    lambda x: n // 4 + 1 < x <= n // 2 + 1,
                )
            ),
            f"(1,a,b,c); a≠b≠c≠a, 1<a≤{n//4 + 1}, {n//4 + 1}<b≤{n//2 + 1}, {n//2 + 1}<c≤{3*n//4 + 1}: {6*(n**3 // 64)}": lambda lst: (
                lst[0] == 1
                and len(set(lst[1:])) == 3
                and can_match_all(
                    lst[1:],
                    lambda x: 1 < x <= n // 4 + 1,
                    lambda x: n // 4 + 1 < x <= n // 2 + 1,
                    lambda x: n // 2 + 1 < x <= 3 * n // 4 + 1,
                )
            ),
            f"(1,a,1,b); a≠b, 1<a,b≤{n//2 + 1}: {comb(n // 2, 2)}": lambda lst: (
                lst[0] == 1 == lst[2]
                and lst[1] != lst[3]
                and 1 < lst[1] <= n // 2 + 1
                and 1 < lst[3] <= n // 2 + 1
            ),
            f"(1,a,1,b); a≠b, 1<a≤{n//2 + 1}, {n//2 + 1}<b≤{3*n//4 + 1}: {n**2 // 8}": lambda lst: (
                lst[0] == 1 == lst[2]
                and lst[1] != lst[3]
                and can_match_all(
                    [lst[1], lst[3]],
                    lambda x: 1 < x <= n // 2 + 1,
                    lambda x: n // 2 + 1 < x <= 3 * n // 4 + 1,
                )
            ),
            "sum": (
                3 * n // 4
                + n // 2
                + n // 4
                + comb(n // 2, 2) * 2
                + (2 * n**2 // 8)
                + (3 * comb(n // 4, 2) * 2)
                + (3 * n**2 // 16)
                + (3 * n**2 // 8)
                + (6 * comb(n // 4, 3))
                + (3 * 2 * comb(n // 4, 2) * (n // 2))
                + (3 * (n // 4) * 2 * comb(n // 4, 2))
                + (6 * (n**3 // 64))
                + (comb(n // 2, 2))
                + (n**2 // 8)
            ),
        }

In [55]:
n = 8
rotate_front_to_back = [f for f in all_collections["Parking functions"].maps if "Mp00293" in str(f)][0]
r = list(lazy_orbits(ParkingFunctions(n), rotate_front_to_back))
lengths = [len(o) for o in r]
hist_values = np.bincount(lengths)
for i in range(1, len(hist_values)):
    conjecture = orbit_size_formula(n, i)
    print(f"Orbit size {i}: {hist_values[i]} instances. Conjectured formula: {conjecture['sum'] if isinstance(conjecture, dict) else conjecture}")
    if not isinstance(conjecture, dict):
        if hist_values[i] < 100:
            for o in r:
                if len(o) == i:
                    print(f"\t{' -> '.join(str(x) for x in o)}")
    else:
        pattern_count = {}
        pattern_matches = {}
        for o in r:
            if len(o) != i:
                continue
            found = None
            for key, pattern in conjecture.items():
                if key == "sum":
                    continue
                if any(pattern(lst[:i]) for lst in o):
                    if found is not None:
                        raise ValueError(f"Found multiple patterns for orbit {o}: {found[:found.index(':')]} and {key[:key.index(':')]}")
                    pattern_count[key] = pattern_count.get(key, 0) + 1
                    found = key
                    
                    
            if found is None:
                raise ValueError(f"Did not find any pattern for orbit {o}")
            if hist_values[i] < 200:
                #print(f"\t{' -> '.join(str(x) for x in o)}: Pattern {found[:found.index(':')]}")
                pattern_matches[found] = pattern_matches.get(found, []) + [o]
        for key in conjecture.keys():
            if key == "sum":
                continue
            print(f"\t{key}. Actual count: {pattern_count.get(key, 0)}")
            if hist_values[i] < 200:
                for o in pattern_matches.get(key, []):
                    print(f"\t\t{' -> '.join(str(x) for x in o)}")


Orbit size 1: 1 instances. Conjectured formula: 1
	[1, 1, 1, 1, 1, 1, 1, 1]
Orbit size 2: 4 instances. Conjectured formula: 4
	[1, 4, 1, 4, 1, 4, 1, 4] -> [4, 1, 4, 1, 4, 1, 4, 1]
	[5, 1, 5, 1, 5, 1, 5, 1] -> [1, 5, 1, 5, 1, 5, 1, 5]
	[1, 2, 1, 2, 1, 2, 1, 2] -> [2, 1, 2, 1, 2, 1, 2, 1]
	[1, 3, 1, 3, 1, 3, 1, 3] -> [3, 1, 3, 1, 3, 1, 3, 1]
Orbit size 3: 0 instances. Conjectured formula: 0
Orbit size 4: 180 instances. Conjectured formula: 180
	(1,1,1,a); 1<a≤7: 6. Actual count: 6
		[1, 1, 1, 5, 1, 1, 1, 5] -> [1, 1, 5, 1, 1, 1, 5, 1] -> [1, 5, 1, 1, 1, 5, 1, 1] -> [5, 1, 1, 1, 5, 1, 1, 1]
		[1, 1, 7, 1, 1, 1, 7, 1] -> [1, 7, 1, 1, 1, 7, 1, 1] -> [7, 1, 1, 1, 7, 1, 1, 1] -> [1, 1, 1, 7, 1, 1, 1, 7]
		[1, 1, 1, 6, 1, 1, 1, 6] -> [1, 1, 6, 1, 1, 1, 6, 1] -> [1, 6, 1, 1, 1, 6, 1, 1] -> [6, 1, 1, 1, 6, 1, 1, 1]
		[1, 4, 1, 1, 1, 4, 1, 1] -> [4, 1, 1, 1, 4, 1, 1, 1] -> [1, 1, 1, 4, 1, 1, 1, 4] -> [1, 1, 4, 1, 1, 1, 4, 1]
		[1, 3, 1, 1, 1, 3, 1, 1] -> [3, 1, 1, 1, 3, 1, 1, 1] -> [1, 1, 1, 3, 1

In [ ]:
colls = widgets.Dropdown(
    options=[(c.name, c) for c in all_collections.values()], description="Collection"
)
maps = widgets.Dropdown(
    options=all_collections[colls.value.name].maps, description="Map"
)


size = widgets.IntSlider(value=3, min=1, max=10, step=1, description="n")

def change_collection(c):
    maps.options = all_collections[c.name].maps
    
colls.observe(lambda change: change_collection(colls.value), names='value')


def evaluate(
    coll,
    func,
    n,
):
    r = list(lazy_orbits(coll.collection(n), func))

    lengths = [len(o) for o in r]
    plt.hist(lengths, bins=range(1, max(lengths) + 2), align="left")
    plt.xlabel("Orbit size")
    plt.ylabel("Count")
    plt.title(f"Orbit sizes for {coll.name} with {func} and n={n}")
    plt.xticks(range(1, max(lengths) + 1))
    plt.grid(axis="y")
    plt.yscale('log')
    plt.show()
    hist_values = np.bincount(lengths)
    for i in range(1, len(hist_values)):
        print(f"Orbit size {i}: {hist_values[i]} instances")
        if hist_values[i] < 200:
            for o in r:
                if len(o) == i:
                    print(f"\t{' -> '.join(str(x) for x in o)}")

interact_manual(
    evaluate,
    coll=colls,
    func=maps,
    n=size,
)

interactive(children=(Dropdown(description='Collection', options=(('Permutations', CollectionWithMapsAndStats(…

<function __main__.evaluate(coll, func, n)>